In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from from_laz_to_csv import laz_to_df

CSV_PATH = "toulouse_capitole.laz"

df = laz_to_df(input_path=CSV_PATH, max_rows=1_000_000)
df = df.drop(['synthetic', 'key_point', 'withheld', 'overlap', 'edge_of_flight_line', 'scanner_channel', 'scan_direction_flag', 'user_data', 'point_source_id', 'gps_time'], axis=1)
# df = df[df['classification'].isin([6, 2, 5, 1])]
df_full = df

In [ ]:
df.describe()

In [ ]:
from from_laz_to_csv import sort_points_by_proximity
df = sort_points_by_proximity(df=df,x=5.745318e+07, y=6.279500e+08, z=1.547453e+04)

In [ ]:
df = df.head(300_000)
df

In [8]:
y = df.classification
X = df.drop('classification', axis=1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3) 
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
from main import process_and_visualize_dataframe

# Préparer les données pour la visualisation
X_test['classification'] = y_test.values  # Ajouter les classifications correctes
process_and_visualize_dataframe(X_test, y_test.values, y_pred)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


# Graphique de la matrice de confusion
plt.figure(figsize=(5, 5))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap='Blues', fmt=".3g")
plt.title('Matrice de Confusion')
plt.xlabel('Prédiction')
plt.ylabel('Vraie Valeur')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X, y)

print("Accuracy_score du modèle : ", model.score(X, y))


feature_importances_ = model.feature_importances_
feature = pd.DataFrame({'Features': X.columns, 'Importance': feature_importances_}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature['Features'], feature['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.show()